# Auto-Sklearn Homework 2


In [1]:
# !pip uninstall --yes nvidia-ml-py3
# !pip uninstall --yes pandas-profiling
# !pip uninstall --yes scipy
# !pip install ydata-profiling
# !pip install auto-sklearn

# 1. uninstall all affected packages
# !pip uninstall -y Cython scipy pyparsing scikit_learn imbalanced-learn mlxtend yellowbrick

# 2. install packages to be downgraded
# !pip install Cython==0.29.36 scipy==1.9 pyparsing==2.4

# 3. install older scikit-learn disregarding its dependencies
# !pip install scikit-learn==0.24.2 --no-build-isolation

# 4. finally install auto-sklearn
# !pip install auto-sklearn

# 5. then, try load

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_table('../../../artificial_train.data', sep=' ', header=None)
train = train.iloc[:, :-1]

y_train = pd.read_table('../../../artificial_train.labels', sep=' ', header=None)
y_train.rename(columns={0: 'label'}, inplace=True)

train = pd.concat([train, y_train], axis=1)

X_test = pd.read_table('../../../artificial_test.data', sep=' ', header=None)
X_test = X_test.iloc[:, :-1]


# X_train, X_val, y_train, y_val = train_test_split(train.iloc[:, :-1], train.iloc[:, -1], test_size=0.15, stratify=train.iloc[:, -1], random_state=42)
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]

In [3]:
pd.set_option('display.width', 120)

In [4]:
complete_df = pd.concat([y_train, X_train], axis=1)

In [5]:
complete_df.describe(include='all')

,label,0,1,2,3,4,5,6,7,8,...,490,491,492,493,494,495,496,497,498,499
count,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.00000,481.722500,483.452500,510.166000,483.384500,501.612500,479.259000,480.109500,476.565000,486.793500,...,482.458500,478.811500,486.356500,496.565500,493.49950,510.893000,478.219500,483.309000,507.977000,490.266000
std,1.00025,6.421769,30.186294,38.899165,9.059895,41.389418,6.795956,40.575925,1.384461,15.043836,...,20.890033,4.011735,23.967366,127.635442,34.81902,37.459353,5.880613,13.559847,37.224297,25.825273
min,-1.00000,462.000000,381.000000,370.000000,453.000000,371.000000,459.000000,334.000000,471.000000,430.000000,...,407.000000,463.000000,391.000000,130.000000,368.00000,398.000000,457.000000,435.000000,363.000000,403.000000
25%,-1.00000,477.000000,464.000000,485.000000,477.000000,475.000000,475.000000,452.750000,476.000000,477.000000,...,468.000000,476.000000,471.000000,404.000000,470.00000,486.000000,474.000000,474.000000,482.000000,473.000000
50%,0.00000,482.000000,483.000000,510.500000,483.000000,500.000000,479.000000,480.000000,477.000000,487.000000,...,482.000000,479.000000,486.000000,504.000000,492.00000,511.000000,478.000000,483.000000,508.000000,490.000000
75%,1.00000,486.000000,503.000000,536.000000,490.000000,528.000000,484.000000,506.250000,477.000000,496.250000,...,496.000000,481.000000,502.000000,586.000000,517.00000,535.000000,482.000000,492.000000,533.000000,507.250000
max,1.00000,503.000000,600.000000,654.000000,519.000000,688.000000,505.000000,611.000000,481.000000,536.000000,...,549.000000,497.000000,566.000000,920.000000,615.00000,661.000000,500.000000,535.000000,644.000000,583.000000


In [6]:
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.metrics import balanced_accuracy

In [7]:
settings = {
  "time_left_for_this_task": 1500,
  "seed": 42,
  "metric": balanced_accuracy,
  "n_jobs": 4,
}

resampling_strategy = "cv"

resampling_strategy_arguments = {
    "train_size": 0.75,     # The size of the training set
    "shuffle": True,        # Whether to shuffle before splitting data
    "folds": 7              # Used in 'cv' based resampling strategies
}

In [8]:
askl1 = AutoSklearnClassifier(
    **settings,
    resampling_strategy=resampling_strategy,
    resampling_strategy_arguments=resampling_strategy_arguments,
)
askl1.fit(X_train, y_train, dataset_name="hw2")

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      metric=balanced_accuracy, n_jobs=4,
                      per_run_time_limit=600, resampling_strategy='cv',
                      resampling_strategy_arguments={'folds': 7,
                                                     'shuffle': True,
                                                     'train_size': 0.75},
                      seed=42, time_left_for_this_task=1500)

In [10]:
import sklearn

def scoring_function(estimator):
    predictions = estimator.predict(X_train)
    return sklearn.metrics.balanced_accuracy_score(y_train, predictions)

In [12]:
from pprint import pprint

print(f"Auto-sklearn 1.0 | results = {scoring_function(askl1)}")
print(f"Selected `resampling-strategy` = {askl1.resampling_strategy}")
print(f"Selected `resampling-strategy-arguments` = {askl1.resampling_strategy_arguments}")

print(askl1.sprint_statistics())

leaderboard = askl1.leaderboard(sort_by="model_id", ensemble_only=True)
print(leaderboard)

pprint(askl1.show_models())

# print(askl1.cv_results_)

Auto-sklearn 1.0 | results = 1.0
Selected `resampling-strategy` = cv
Selected `resampling-strategy-arguments` = {'train_size': 0.75, 'shuffle': True, 'folds': 7}
auto-sklearn results:
  Dataset name: hw2
  Metric: balanced_accuracy
  Best validation score: 0.885487
  Number of target algorithm runs: 25
  Number of successful target algorithm runs: 15
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 4

          rank  ensemble_weight                 type      cost    duration
model_id                                                                  
3            2             0.32        random_forest  0.117020  350.695145
4            3             0.06  k_nearest_neighbors  0.118509  292.926442
6            4             0.12        random_forest  0.122521  363.230102
9            1             0.40          extra_trees  0.114513  339.143444
23           5           

In [13]:
test_probs = askl1.predict_proba(X_test)
test_probs = test_probs[:, 1]

test_probs = pd.DataFrame(test_probs)
test_probs.columns = ['313325_313342']
test_probs.head(2)

,313325_313342
0,0.193041
1,0.073548


In [15]:
settings

{'time_left_for_this_task': 1500,
 'seed': 42,
 'metric': balanced_accuracy,
 'n_jobs': 4}

In [14]:
resampling_strategy_arguments

{'train_size': 0.75, 'shuffle': True, 'folds': 7}

In [16]:
test_probs.to_csv('../../313325_313342_artificial_automl_prediction.txt', index=False)